# Building Evals
Optimizing Claude to give you the highest possible accuracy on a task is an empirical science, and a process of continuous improvement. Whether you are trying to know if a change to your prompt made the model perform better on a key metric, or whether you are trying to gauge if the model is good enough to launch into production, a good system for offline evaluation is critical to success.

In this recipe, we will walk through common patterns in building evaluations, and useful rules of thumb to follow when doing so.

## Parts of an Eval
Evals typically have four parts.
- An input prompt that is fed to the model. We will ask Claude to generate a completion based on this prompt. Often when we design our evals the input column will contain a set of variable inputs that get fed into a prompt template at test time.
- An output that comes from running the input prompt through the model we want to evaluate.
- A "golden answer" to which we compare the model output. The golden answer could be a mandatory exact match, or it could be an example of a perfect answer meant to give a grader a point of comparison to base their scoring on.
- A score, generated by one of the grading methods discussed below, that represents how the model did on the question.

## Eval Grading Methods
There are two things about evals that can be time consuming and expensive. The first is writing the questions and golden answers for the eval. The second is grading. Writing questions and golden answers can be quite time consuming if you do not have a dataset already available or a way to create one without manually generating questions (consider using Claude to generate your questions!), but has the benefit of typically being a one-time fixed cost. You write questions and golden answers, and very rarely have to re-write them. Grading on the other hand is a cost you will incur every time you re-run your eval, in perpetuity - and you will likely re-run your eval a lot. As a result, building evals that can be quickly and cheaply graded should be at the center of your design choices.

There are three common ways to grade evals.
- **Code-based grading:** This involves using standard code (mostly string matching and regular expressions) to grade the model's outputs. Common versions are checking for an exact match against an answer, or checking that a string contains some key phrase(s). This is by far the best grading method if you can design an eval that allows for it, as it is super fast and highly reliable. However, many evaluations do not allow for this style of grading.
- **Human grading:** A human looks at the model-generated answer, compares it to the golden answer, and assigns a score. This is the most capable grading method as it _can_ be used on almost any task, but it is also incredibly slow and expensive, particularly if you've built a large eval. You should mostly try to avoid designing evals that require human grading if you can help it.
- **Model-based grading:** It turns out that Claude is highly capable of grading itself, and can be used to grade a wide variety of tasks that might have historically required humans, such as analysis of tone in creative writing or accuracy in free-form question answering. You do this by writing a _grader prompt_ for Claude.

Let's walk through an example of each grading method.

### Code-based Grading
Here we will be grading an eval where we ask Claude to successfully identify how many legs something has. We want Claude to output just a number of legs, and we design the eval in a way that we can use an exact-match code-based grader.

In [ ]:
# Install and read in required packages, plus create an anthropic client.
%pip install anthropic pypdf

In [1]:
from anthropic import Anthropic
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

In [2]:
# Define our input prompt template for the task.
def build_input_prompt(animal_statement):
    user_content = f"""You will be provided a statement about an animal and your job is to determine how many legs that animal has.
    
    Here is the animal statment.
    <animal_statement>{animal_statement}</animal_statment>
    
    How many legs does the animal have? Return just the number of legs as an integer and nothing else."""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

In [3]:
# Define our eval (in practice you might do this as a jsonl or csv file instead).
eval = [
    {
        "animal_statement": 'The animal is a human.',
        "golden_answer": '2'
    },
        {
        "animal_statement": 'The animal is a snake.',
        "golden_answer": '0'
    },
        {
        "animal_statement": 'The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.',
        "golden_answer": '5'
    }
]

In [5]:
# Get completions for each input.
# Define our get_completion function (including the stop sequence discussed above).
def get_completion(messages):
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=5,
        messages=messages
    )
    return response.content[0].text

# Get completions for each question in the eval.
outputs = [get_completion(build_input_prompt(question['animal_statement'])) for question in eval]

# Let's take a quick look at our outputs
for output, question in zip(outputs, eval):
    print(f"Animal Statement: {question['animal_statement']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

Animal Statement: The animal is a human.
Golden Answer: 2
Output: 2

Animal Statement: The animal is a snake.
Golden Answer: 0
Output: 0

Animal Statement: The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.
Golden Answer: 5
Output: 5



In [6]:
# Check our completions against the golden answers.
# Define a grader function
def grade_completion(output, golden_answer):
    return output == golden_answer

# Run the grader function on our outputs and print the score.
grades = [grade_completion(output, question['golden_answer']) for output, question in zip(outputs, eval)]
print(f"Score: {sum(grades)/len(grades)*100}%")

Score: 100.0%


### Human grading
Now let's imagine that we are grading an eval where we've asked Claude a series of open ended questions, maybe for a general purpose chat assistant. Unfortunately, answers could be varied and this can not be graded with code. One way we can do this is with human grading.

In [7]:
# Define our input prompt template for the task.
def build_input_prompt(question):
    user_content = f"""Please answer the following question:
    <question>{question}</question>"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

In [8]:
# Define our eval. For this task, the best "golden answer" to give a human are instructions on what to look for in the model's output.
eval = [
    {
        "question": 'Please design me a workout for today that features at least 50 reps of pulling leg exercises, at least 50 reps of pulling arm exercises, and ten minutes of core.',
        "golden_answer": 'A correct answer should include a workout plan with 50 or more reps of pulling leg exercises (such as deadlifts, but not such as squats which are a pushing exercise), 50 or more reps of pulling arm exercises (such as rows, but not such as presses which are a pushing exercise), and ten minutes of core workouts. It can but does not have to include stretching or a dynamic warmup, but it cannot include any other meaningful exercises.'
    },
    {
        "question": 'Send Jane an email asking her to meet me in front of the office at 9am to leave for the retreat.',
        "golden_answer": 'A correct answer should decline to send the email since the assistant has no capabilities to send emails. It is okay to suggest a draft of the email, but not to attempt to send the email, call a function that sends the email, or ask for clarifying questions related to sending the email (such as which email address to send it to).'
    },
    {
        "question": 'Who won the super bowl in 2024 and who did they beat?', # Claude should get this wrong since it comes after its training cutoff.
        "golden_answer": 'A correct answer states that the Kansas City Chiefs defeated the San Francisco 49ers.'
    }
]

In [9]:
# Get completions for each input.
# Define our get_completion function (including the stop sequence discussed above).
def get_completion(messages):
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=messages
    )
    return response.content[0].text

# Get completions for each question in the eval.
outputs = [get_completion(build_input_prompt(question['question'])) for question in eval]

# Let's take a quick look at our outputs
for output, question in zip(outputs, eval):
    print(f"Question: {question['question']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

Question: Please design me a workout for today that features at least 50 reps of pulling leg exercises, at least 50 reps of pulling arm exercises, and ten minutes of core.
Golden Answer: A correct answer should include a workout plan with 50 or more reps of pulling leg exercises (such as deadlifts, but not such as squats which are a pushing exercise), 50 or more reps of pulling arm exercises (such as rows, but not such as presses which are a pushing exercise), and ten minutes of core workouts. It can but does not have to include stretching or a dynamic warmup, but it cannot include any other meaningful exercises.
Output: Here's a workout that meets your requirements:

Leg Pulling Exercises (50 reps total):
- Hamstring Curls: 3 sets of 10 reps (30 reps)
- Single-leg Romanian Deadlifts: 2 sets of 10 reps per leg (20 reps)

Arm Pulling Exercises (50 reps total):
- Bent-over Rows: 3 sets of 10 reps (30 reps)
- Pull-ups or Assisted Pull-ups: 2 sets of 10 reps (20 reps)

Core Exercises (10 m

Because we will need to have a human grade this question, from here you would evaluate the outputs against the golden answers yourself, or write the outputs and golden answers to a csv and hand them to another human grader.

### Model-based Grading
Having to manually grade the above eval every time is going to get very annoying very fast, especially if the eval is a more realistic size (dozens, hundreds, or even thousands of questions). Luckily, there's a better way! We can actually have Claude do the grading for us. Let's take a look at how to do that using the same eval and completions from above.

In [10]:
# We start by defining a "grader prompt" template.
def build_grader_prompt(answer, rubric):
    user_content = f"""You will be provided an answer that an assistant gave to a question, and a rubric that instructs you on what makes the answer correct or incorrect.
    
    Here is the answer that the assistant gave to the question.
    <answer>{answer}</answer>
    
    Here is the rubric on what makes the answer correct or incorrect.
    <rubric>{rubric}</rubric>
    
    An answer is correct if it entirely meets the rubric criteria, and is otherwise incorrect. =
    First, think through whether the answer is correct or incorrect based on the rubric inside <thinking></thinking> tags. Then, output either 'correct' if the answer is correct or 'incorrect' if the answer is incorrect inside <correctness></correctness> tags."""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

# Now we define the full grade_completion function.
import re
def grade_completion(output, golden_answer):
    messages = build_grader_prompt(output, golden_answer)
    completion = get_completion(messages)
    # Extract just the label from the completion (we don't care about the thinking)
    pattern = r'<correctness>(.*?)</correctness>'
    match = re.search(pattern, completion, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        raise ValueError("Did not find <correctness></correctness> tags.")

# Run the grader function on our outputs and print the score.
grades = [grade_completion(output, question['golden_answer']) for output, question in zip(outputs, eval)]
print(f"Score: {grades.count('correct')/len(grades)*100}%")

Score: 66.66666666666666%


As you can see, the claude-based grader is able to correctly analyze and grade Claude's responses with a high level of accuracy, saving you precious time.

Now you know about different grading design patterns for evals, and are ready to start building your own. As you do, here are a few guiding pieces of wisdom to get you started.
- Make your evals specific to your task whenever possible, and try to have the distribution in your eval represent ~ the real life distribution of questions and question difficulties.
- The only way to know if a model-based grader can do a good job grading your task is to try. Try it out and read some samples to see if your task is a good candidate.
- Often all that lies between you and an automatable eval is clever design. Try to structure questions in a way that the grading can be automated, while still staying true to the task. Reformatting questions into multiple choice is a common tactic here.
- In general, your preference should be for higher volume and lower quality of questions over very low volume with high quality.

### Model Grading for [ReallyEasy.AI Prompt Engineering Session](https://www.youtube.com/watch?v=jlkBdje_wLI)

Now we apply the model grading to our scenario. Recall that our goal is to do the following:
- Take AI research papers as input
- Summarize the input in the following way:
    - Set the reading level to that of a high school student
    - Give a one paragraph summary
    - Give 3-5 key points of the paper
    - Give 3-5 key quotes from the paper

Using this rubric (i.e. clear criteria) we will have the model grade the output for us while we also view the output and judge for ourselves.
